# Import libraries

In [ ]:
import os
import csv
import time
import pickle
import numpy as np
import tensorflow as tf

from models.deep_crnn_model import *
from models.model_builders.create_dataset import *

# Model parameters

In [ ]:
csv_files_train = "../data/iamHandwriting/train.csv"

output_model_dir = "../tf_output/estimator/"
output_preds_dir = "../tf_output/prediction/"
output_graph_dir = "../tf_output/graph/"

batch_size=64

learning_rate=1e-3
learning_decay_rate=0.95
learning_decay_steps=5000

input_shape=(129, 369) #iam shape
# input_shape=(117, 1669) # bentham shape1
# input_shape=(181, 1833) # bentham shape2

optimizer='adam'

# alphabet=" !\"#&'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXY[]_abcdefghijklmnopqrstuvwxyz|"
alphabet="abcdefghijklmnopqrstuvwxyz"

csv_delimiter='\t'

# Run model

** Create model ops **

In [ ]:
input_tensor = tf.placeholder(tf.float32, [None, input_shape[0], input_shape[1], 1])
labels = tf.placeholder(tf.string, [None])

out = deep_crnn(input_tensor, labels, input_shape, alphabet, batch_size)
train_op, loss_ctc, CER, accuracy, predictions_dict = out

** Load dataset **

In [ ]:
out = create_iterator(csv_files_train, input_shape, batch_size)
dataset, iterator, next_batch, datasize = out

** Predict model **

In [ ]:
saver = tf.train.Saver()

labls = []
losses = []
cers = []
accs = []
preddicts = []

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, output_model_dir+"/model.ckpt")
    
    writer = tf.summary.FileWriter(output_graph_dir+"/preds", sess.graph)
    n_batches = int(datasize / train_batch_size)
    print("Starting epoch")
    sess.run(iterator.initializer)
    for j in range(n_batches):
        input_tensor_b, labels_b = sess.run(next_batch)
        
        try:
            cer, acc, loss, preddict = sess.run([CER, accuracy, loss_ctc, predictions_dict],
                         feed_dict={input_tensor: input_tensor_b, labels: labels_b})
            labls.append(labels_b)
            losses.append(loss)
            cers.append(cer)
            accs.append(accs)
            preddicts.append(preddict)

            pickle.dump([labls, accs, losses, cers, preddicts], open(output_preds_dir+"/preds.pkl", "wb"))
            print('batch: {0}, CER: {1}, accuracy: {2}, loss: {3}'.format(j, cer, acc, loss))
        except:
            labls.append([])
            losses.append([])
            cers.append([])
            accs.append([])
            preddicts.append([])

            pickle.dump([labls, accs, losses, cers, preddicts], open(output_preds_dir+"/preds.pkl", "wb"))
            print("Error at ", j)

    print('Total time: {0} seconds'.format(time.time() - start_time))